# Preprocessing D2.1
In this notebook the main objective is to expand the events that are represented as a start date and an end date, to get them monhtly expressed.

Also here we will add a prefix to each event depending on their type to allow us to indentify the type easily a posteriori and to avoid the eventtype column.

In [2]:
%matplotlib inline
from dateutil import relativedelta
import datetime as dt
import matplotlib.pylab as plt
import numpy as np
import os
import pandas as pd
from sklearn.externals import joblib

In [3]:
events = joblib.load('events.pickle')
events['dat'] = pd.to_datetime(events['dat'], errors='coerce', format ='%Y%m')
events['dbaixa'] = pd.to_datetime(events['dbaixa'], errors='coerce', format='%Y%m')
print(events.index.size)
display(events.head())
pob = joblib.load('p.pickle')
display(pob.head())

47347863


,id,dat,eventtype,code,dbaixa,val
0,6,2012-11-01,SMOKING,NaN,2013-02-01,1.0
1,6,2012-11-01,MEASUREMENT,ALDIA,NaT,1.0
2,6,2012-11-01,MEASUREMENT,ALHAB,NaT,5.0
3,6,2012-11-01,MEASUREMENT,ALRIS,NaT,1.0
4,6,2012-11-01,MEASUREMENT,ALSET,NaT,6.0


,id,sexe,qmedea,dnaix
0,8636519,D,U5,1959
1,10937990,H,U1,1937
2,1524971,D,U3,1933
3,9323816,H,U3,1950
4,6749464,H,U3,1938


# Moving Events

In [5]:
dict_sidiap = dict(
    VISIT= "v",
    VACCINE= "a",
    DRUG_DISP= "f",
    DRUG_PRES= "p",
    SMOKING= "s",
    EXPLORATION= "e",
    LABORATORY= "l",
    MEASUREMENT= "m",
    SICK_LEAVE= "b",
    DX= "d"
)
m = events['code'].isnull()
m1 = events['val'].isnull()
events['new_code'] = events['eventtype'].map(dict_sidiap)+'_'+events['code']
events.loc[m&~m1,'new_code'] = events['eventtype'].map(dict_sidiap)+'_'+events['val'].astype(str)
events.loc[m1&m,'new_code'] = events['eventtype'].map(dict_sidiap)
display(events.head())

,id,dat,eventtype,code,dbaixa,val,new_code
0,6,2012-11-01,SMOKING,NaN,2013-02-01,1.0,s_1.0
1,6,2012-11-01,MEASUREMENT,ALDIA,NaT,1.0,m_ALDIA
2,6,2012-11-01,MEASUREMENT,ALHAB,NaT,5.0,m_ALHAB
3,6,2012-11-01,MEASUREMENT,ALRIS,NaT,1.0,m_ALRIS
4,6,2012-11-01,MEASUREMENT,ALSET,NaT,6.0,m_ALSET


In [5]:
events[events['eventtype'] == 'SICK_LEAVE'].head()

,id,dat,eventtype,code,dbaixa,val,new_code
310,124,2006-02-01,SICK_LEAVE,NaN,2006-03-01,NaN,b
324,124,2007-05-01,SICK_LEAVE,NaN,2007-05-01,NaN,b
837,174,2010-09-01,SICK_LEAVE,NaN,2010-10-01,NaN,b
944,174,2014-02-01,SICK_LEAVE,NaN,2014-04-01,NaN,b
3159,779,2010-05-01,SICK_LEAVE,NaN,2010-07-01,NaN,b


# Generating TYPE1

In [6]:
# Event type counters
events_filtre = list({'VISIT', 'DRUG_DISP', 'EXPLORATION', 'VACCINE','LABORATORY','MEASUREMENT'})
m = events['eventtype'].isin(events_filtre)
events_type1 = events.loc[m, ['id','dat','val','new_code']]
display(events_type1.head())
joblib.dump(events_type1, 'numerical_events.pickle', compress=3)

,id,dat,val,new_code
1,6,2012-11-01,1.0,m_ALDIA
2,6,2012-11-01,5.0,m_ALHAB
3,6,2012-11-01,1.0,m_ALRIS
4,6,2012-11-01,6.0,m_ALSET
5,6,2012-11-01,138.0,m_EK201


['numerical_events.pickle']

# Generating DEATHs

In [ ]:
m = events['eventtype']=='DEATH'
events_death = events.loc[m,['id','dat']]
display(events_death.head())
joblib.dump(events_death, 'deaths_events.pickle', compress=3)

# Generating TYPE2

In [6]:
events_cop = events.copy()

In [26]:
events = events_cop

In [27]:
print(events.head())
events_filtre = list({'DX', 'DRUG_PRES', 'SMOKING', 'SICK_LEAVE'})
m = events['dbaixa'].isnull()
m1 = events['eventtype'].isin(events_filtre)

   id        dat    eventtype   code     dbaixa  val new_code
0   6 2012-11-01      SMOKING    NaN 2013-02-01  1.0    s_1.0
1   6 2012-11-01  MEASUREMENT  ALDIA        NaT  1.0  m_ALDIA
2   6 2012-11-01  MEASUREMENT  ALHAB        NaT  5.0  m_ALHAB
3   6 2012-11-01  MEASUREMENT  ALRIS        NaT  1.0  m_ALRIS
4   6 2012-11-01  MEASUREMENT  ALSET        NaT  6.0  m_ALSET


In [28]:
events.loc[(m & m1), 'dbaixa'] = pd.to_datetime('20161201', format='%Y%m%d', errors='coerce')

In [29]:
events_to_expand = events.loc[m1, ['id','dat','new_code','dbaixa']]
display(events_to_expand.head())
print(events_to_expand.size)
print(events_to_expand.index.size)

,id,dat,new_code,dbaixa
0,6,2012-11-01,s_1.0,2013-02-01
31,6,2013-02-01,s_2.0,2016-12-01
37,6,2013-02-01,d_I10,2016-12-01
41,6,2013-04-01,p_C09AA02,2016-12-01
100,6,2015-05-01,p_J01CR02,2015-05-01


27244028
6811007


In [30]:
expanded = pd.DataFrame()

#m = events_to_expand['dat'] == events_to_expand['dbaixa']
#events_to_expand = events_to_expand.loc[~m]
events_to_expand.reset_index(inplace = True)
display(events_to_expand.head())
print(events_to_expand.index.size)

time_index = pd.DataFrame(sorted(events['dat'].unique()), columns=['dat'])
#del events

print(time_index)

def expanding_empty_periods(data):
    df_expanded = []
    for i in data['id'].unique():
        ti = time_index.copy()
        ti['id'] = i
        df_expanded.append(ti)
    print(len(df_expanded))
    
    df_expanded = pd.concat(df_expanded, axis=0)
    
    df2 = df_expanded.merge(data, on = ['id'])
    m = (df2['dat_x'] >= df2['dat_y']) & (df2['dat_x'] <= df2['dbaixa'])
    df2 = df2.loc[m]
    #display(df2.head())
    df2.drop(columns=['dat_y', 'dbaixa'], inplace = True)
    return df2
    #joblib.dump(df2, 'chunk'+str(id)+'.pickle', compress=3)

N = 100000
i = 0
end = True
TAM = events_to_expand.index.size
while end == True:
    print('Iteration ', N)
    dat = events_to_expand.loc[i:N-1]

    x = expanding_empty_periods(dat)
    #del dat
    expanded = expanded.append(x)
    del x
 

    if N == TAM:
        print('We have finished')
        end = False
    elif N > TAM:
        N = TAM
    else:
        i = N
        N = N+100000

joblib.dump(expanded, 'expanded_events.pickle', compress=3)

,index,id,dat,new_code,dbaixa
0,0,6,2012-11-01,s_1.0,2013-02-01
1,31,6,2013-02-01,s_2.0,2016-12-01
2,37,6,2013-02-01,d_I10,2016-12-01
3,41,6,2013-04-01,p_C09AA02,2016-12-01
4,100,6,2015-05-01,p_J01CR02,2015-05-01


6811007
           dat
0   2006-01-01
1   2006-02-01
2   2006-03-01
3   2006-04-01
4   2006-05-01
5   2006-06-01
6   2006-07-01
7   2006-08-01
8   2006-09-01
9   2006-10-01
10  2006-11-01
11  2006-12-01
12  2007-01-01
13  2007-02-01
14  2007-03-01
15  2007-04-01
16  2007-05-01
17  2007-06-01
18  2007-07-01
19  2007-08-01
20  2007-09-01
21  2007-10-01
22  2007-11-01
23  2007-12-01
24  2008-01-01
25  2008-02-01
26  2008-03-01
27  2008-04-01
28  2008-05-01
29  2008-06-01
..         ...
102 2014-07-01
103 2014-08-01
104 2014-09-01
105 2014-10-01
106 2014-11-01
107 2014-12-01
108 2015-01-01
109 2015-02-01
110 2015-03-01
111 2015-04-01
112 2015-05-01
113 2015-06-01
114 2015-07-01
115 2015-08-01
116 2015-09-01
117 2015-10-01
118 2015-11-01
119 2015-12-01
120 2016-01-01
121 2016-02-01
122 2016-03-01
123 2016-04-01
124 2016-05-01
125 2016-06-01
126 2016-07-01
127 2016-08-01
128 2016-09-01
129 2016-10-01
130 2016-11-01
131 2016-12-01

[132 rows x 1 columns]
Iteration  100000
2661
Iteration  2000

['expanded_events.pickle']

In [31]:
expanded.head()

,dat_x,id,index,new_code
820,2012-11-01,6,0,s_1.0
830,2012-12-01,6,0,s_1.0
840,2013-01-01,6,0,s_1.0
850,2013-02-01,6,0,s_1.0
851,2013-02-01,6,31,s_2.0


In [32]:
expanded.rename(columns={'dat_x': 'dat'}, inplace = True)
expanded.reset_index(drop = True, inplace = True)
expanded.drop(columns=['index'], inplace = True)
display(expanded.head())

,dat,id,new_code
0,2012-11-01,6,s_1.0
1,2012-12-01,6,s_1.0
2,2013-01-01,6,s_1.0
3,2013-02-01,6,s_1.0
4,2013-02-01,6,s_2.0


In [36]:
m2 = events['id'] == 6
events.loc[m1 & m2].sort_values('dat')

,id,dat,eventtype,code,dbaixa,val,new_code
0,6,2012-11-01,SMOKING,NaN,2013-02-01,1.0,s_1.0
31,6,2013-02-01,SMOKING,NaN,2016-12-01,2.0,s_2.0
37,6,2013-02-01,DX,I10,2016-12-01,NaN,d_I10
41,6,2013-04-01,DRUG_PRES,C09AA02,2016-12-01,NaN,p_C09AA02
100,6,2015-05-01,DRUG_PRES,J01CR02,2015-05-01,NaN,p_J01CR02
101,6,2015-05-01,DX,K04.7,2015-07-01,NaN,d_K04.7
102,6,2015-05-01,DRUG_PRES,M01AE17,2015-05-01,NaN,p_M01AE17
105,6,2015-05-01,DRUG_PRES,N02BB02,2015-05-01,NaN,p_N02BB02
108,6,2015-08-01,DX,R05,2016-08-01,NaN,d_R05
112,6,2016-02-01,DX,R10.1,2016-12-01,NaN,d_R10.1


In [34]:
expanded.drop_duplicates(inplace = True)

In [40]:
m2 = expanded['id'] == 6
expanded.loc[m2].sort_values(['new_code', 'dat'])

,dat,id,new_code
5,2013-02-01,6,d_I10
7,2013-03-01,6,d_I10
9,2013-04-01,6,d_I10
12,2013-05-01,6,d_I10
15,2013-06-01,6,d_I10
18,2013-07-01,6,d_I10
21,2013-08-01,6,d_I10
24,2013-09-01,6,d_I10
27,2013-10-01,6,d_I10
30,2013-11-01,6,d_I10


In [41]:
joblib.dump(expanded, 'pickles/final_events_periods.pickle', compress=3)

['pickles/final_events_periods.pickle']

In [42]:
x = expanded.groupby(expanded['new_code']).size().to_frame('size')
display(x.sort_values('size', ascending  = False))

,size
new_code,
s_0.0,4011316
d_Z00.1,2014149
s_1.0,1891217
p_N02BE01,1349132
p_A02BC01,1237231
d_M54.5,1177472
d_I10,1118726
d_K02,966597
s_2.0,857145
